### Prep:

* Resample all inputs see 1-Resample Input Data.ipynb
* Rasterize fires and organize by time period see 2-RasterizeFirePolygons_OrganizeData.ipynb

### Data Structure:
Data should be structured as follows:

* Independent Vars 2012-2016
    * aet
        * aet-201201.tif
        * ...
        * aet-201612.tif
    * cwd
        * cwd-201201.tif
        * ...
        * cwd-201612.tif
    * ppt
        * ppt-201201.tif
        * ...
        * ppt-201612.tif
    * ...
* Independent Vars 2017-2021
    * ...
* Dependent Var 2012-2016
    * Fire-2012-2016
* Dependent Var 2017-2021
    * Fire-2017-2021

In [1]:
import tsraster.prep as tr
import tsraster.random as random
import tsraster.model as model
import tsraster.calculate as ca
import numpy as np

import tsraster.model  as md
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
from math import ceil
from tsraster.prep import set_df_mindex
import copy

In [2]:
########################### 

In [3]:
#construct poisson disk mask, which masks out all pixels not selected by raster mask
'''
Create raster of cells to be selected (populated as ones) in a raster of background value zero

:param raster_mask: name of raster mask - provides dimensions for subsample, and also masks unusable areas - 
        Remaining sample area is assumed to be contiguous
:param outFile: path and name of output mask consisting of a rater image with values of 1 for selected pixels, 
        and 0 for all other pixels
:param k: number of attempts to select a point around each reference point before marking it as inactive
:param r: minimum distance (in raster cells) between selected points 
:return:  list which includes an array of all masked & unnmasked cells, and a dictionary of all selected points.
            Also saves the a raster consisting of 0s for all non-selected points, and 1s for all selected points
            to the outFile location.
'''


rasterMask = random.Poisson_Subsample(raster_mask = r"../Data\Examples\buffer\StatePoly_buf.tif",
                                      outFile = r"../Data\Examples\diskTest.tif",
                                      k=50, 
                                      r=5)[0]

In [4]:
#combine all data that is invariant over time into single dataFrame 
#(e.g., elevation, rate of lightning strikes, fire station locations)

'''Combines set of rasters to single dataFrame based on csv that lists all desired files 
    and column names to use in dataframe for data corresponding to each raster
    
    :param csvPath: path to csv of filepaths (in column "FilePath") and desired label names for values in each raster (in column "DataName")
    :param outPath: path to ouput file name and location
    :return: dataFrame consisting of all desired data with previously selected labels.
 '''

invar_Data = tr.multi_image_to_dataframe("C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Actual/Invar_FileList.csv", "C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Actual/")
invar_Data.head()

,pixel_id,Campground,FireStation_Dist,Airfield_Dist,City_Bounds,NPS_Bounds,Lightning,Elev
0,0,-1,121,78,-1,-1,0,-15
1,1,-1,-23,-66,-1,-1,0,-15
2,2,-1,95,49,-1,-1,0,-15
3,3,-1,-38,-88,-1,-1,0,-15
4,4,-1,91,34,-1,-1,0,-15


In [5]:
#conduct climate feature extraction across all years of interest


'''
Extracts summary statistics(features) from multiYear datasets within moving window, across years
Outputs a series of annual dataFrames as CSV files

:param startYear: year on which to start feature extraction
:param endYear: year on which to end feature extraction
:param featureData_Path: file path to data from which to extract features
:param feature_params: summary statistics(features) to extract from data within each window
:param invar_Data: year-invariate data to join with extracted feature data on an annual scale
:param out_Path: file path to location at which extracted features should be output as a csv
:param window_length: length of window within which to extract features
:param window_offset: number of years by which features pertaining to each year are offset from that year
:param mask:  mask to apply to data prior to feature extraction
:return: no return.  instead, feature data relative to each year of interest is saved as a .csv file at the out_Path location
          under the filename FD_Window_XXXX.csv 
'''
ca.multiYear_Window_Extraction(1980, 2016, "C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Actual/Climate/BCM HIST Final 1000m_1950_2016/",
                    feature_params = {"mean": None,"maximum": None}, 
                    invar_Data = invar_Data,
                    out_Path = 'C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/',
                        mask = "C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Examples/buffer/StatePoly_buf.tif" ,
                    window_length = 3,
                    window_offset = 0)

df: C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Actual/Climate/BCM HIST Final 1000m_1950_2016/my_df.csv


Feature Extraction: 100%|████████████████████████| 5/5 [01:06<00:00, 13.57s/it]


features:C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/extracted_features1980_3_prev_offset0.csv
tif:C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/extracted_features1980_3_prev_offset0.tiff
df: C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Actual/Climate/BCM HIST Final 1000m_1950_2016/my_df.csv


Feature Extraction: 100%|████████████████████████| 5/5 [01:07<00:00, 13.88s/it]


features:C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/extracted_features1981_3_prev_offset0.csv
tif:C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/extracted_features1981_3_prev_offset0.tiff
df: C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Actual/Climate/BCM HIST Final 1000m_1950_2016/my_df.csv


...

features:C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/extracted_features2014_3_prev_offset0.csv
tif:C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/extracted_features2014_3_prev_offset0.tiff
df: C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Actual/Climate/BCM HIST Final 1000m_1950_2016/my_df.csv


Feature Extraction: 100%|████████████████████████| 5/5 [01:06<00:00, 13.69s/it]


features:C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/extracted_features2015_3_prev_offset0.csv
tif:C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/extracted_features2015_3_prev_offset0.tiff
df: C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Actual/Climate/BCM HIST Final 1000m_1950_2016/my_df.csv


Feature Extraction: 100%|████████████████████████| 5/5 [01:06<00:00, 13.70s/it]


features:C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/extracted_features2016_3_prev_offset0.csv
tif:C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/extracted_features2016_3_prev_offset0.tiff


In [6]:
#Assembly of all explanatory variables into annual dataFrames

'''merge additional annually repeating data into feature data, as well as time-invariant data
    Produces annual dataFrames consisting of all explanatory variables that may be incorporated into model
        (Consisting of features extracted from climate data in preceding years, 
        annually repeating data such as estimated housing density,
        and time-invariant data such as rate of lightning strikes or local elevation)
        
    param startYear: year on which to start feature extraction
    param endYear: year on which to end feature extraction
    param other_Data_prefixList: list of file path and portion of filename preceding year for additional Data
    param feature_Data_suffixList: portion of feature data file name that follows year for additional data
    param dataNameList: list of intended data names for additional data
    param outPath: filepath for folder in which the output will be placed
'''
    


tr.annual_Data_Merge(startYear = 1980, 
                     endYear = 2016, 
                  feature_path = "C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/", 
                 invarData_csvPath = "C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Actual/Invar_FileList.csv",
                     other_Data_prefixList = ["C:/Users/Python3/Documents/wildfire_FRAP/Data/Actual/SERGOM_Housing/Interpolated/bhc"],
                 other_Data_suffixList = ["linreg.tif"],
                 dataNameList = ["Housing_Density"],
                 outPath = 'C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/')

In [7]:
#Conversion of fire data into annual dataframes

#converts annual fire raster data into annual CSV files, does some minor reformatting for downstream processing
'''convert annual fire data rasters into annual dataFrames, export as .CSV files
    also does some minor reformatting to prevent problems with downstream processing

    
    #param startYear: year on which to start feature extraction
    #param endYear: year on which to end feature extraction
    #param file_Path: path to target data files (fire data)
    #outPath: filepath for folder in which the output will be placed: 

'''

tr.target_Data_to_csv_multiYear(1980, 2016,
                            "C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Actual/Fires/Rasters/",
                            'C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/')

In [8]:
#Mask All data files (combined_data and target_data) using output of Poisson Disk Masking (or other desired mask)

'''masks annual datasets (combined and fire), and saves the resulting masked data files as CSV files
    Also returns concatenated multiYear files (combined and fire) for subsequent processing - adds in 'year' as a data column
'''
    #param startYear: year on which to begin
    #param endYear: year on which to end
    #param DataLists: csv of files to pull, with the year is the index, 
    #       "combined_Data_Filepaths" as the column of combined data filepaths, and
    #       "target_Data_filePaths" as the column of the target data(i.e. fire) filepaths.
    #param maskFile: filepath to data file used for masking
    #outPath: filepath for folder in which the output will be placed


combined_Data, target_Data = tr.multiYear_Mask(1980, 2016, 
                                               'C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/', 
                                               r"../Data\Examples\diskTest.tif", 
                                               "C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Masked/")

C:\Users\Python3\Anaconda3\envs\ts-raster\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [2]:
#temporary for testing only - read in combined & target data 

combined_Data = pd.read_csv('C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Masked/CD_1980_Masked_2016.csv')
target_Data = pd.read_csv('C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Masked/TD_1980_Masked_2016.csv')
combined_Data.head()

,pixel_id,Unnamed: 0,pixel_id.1,time,aet__maximum,aet__mean,cwd__maximum,cwd__mean,Campground,FireStation_Dist,Airfield_Dist,City_Bounds,NPS_Bounds,Lightning,Elev,Housing_Density,year
0,135537,135537,135537,197801_198012,158.951996,61.840744,37.653000,19.579527,-1,-46,-26,-1,-1,0,91,0,1980
1,135546,135546,135546,197801_198012,133.356003,43.613838,70.831200,36.842278,-1,-30,-56,-1,-1,0,-76,0,1980
2,135569,135569,135569,197801_198012,158.940002,57.745651,43.958801,22.219458,-1,55,15,-1,-1,0,111,0,1980
3,136471,136471,136471,197801_198012,77.573128,28.880606,96.156876,51.681374,-1,-19,111,-1,-1,0,80,1,1980
4,136495,136495,136495,197801_198012,80.501877,25.742798,112.515625,53.565712,-1,-78,-60,-1,-1,0,114,0,1980


In [3]:
#Conduct elastic net regularization

'''Conduct elastic net regressions on data, with k-fold cross-validation conducted independently 
      across both years and pixels. 
      Returns mean model MSE and R2 when predicting fire risk at 
      A) locations outside of the training dataset
      B) years outside of the training dataset
      C) locations and years outside of the training dataset
    Returns a list of objects, consisting of:
    0: Combined_Data file with testing/training groups labeled
    1: Target Data file with testing/training groups labeled
    2: summary dataFrame of MSE and R2 for each model run
        (against holdout data representing either novel locations, novel years, or both)
    3: list of elastic net models for use in predicting Fires in further locations/years
    4: list of list of years not used in model training for each run
'''
#param combined_Data: explanatory factors to be used in predicting fire risk
#param target_Data: observed fire occurrences
#param varsToGroupBy: list of (2) column names from combined_Data & target_Data to be used in creating randomized groups
#    -spatial grouping variable 1st, temporal (annual) grouping variable 2nd
#param groupVars: list of (2) desired column names for the resulting randomized groups
#param testGroups: number of distinct groups into which data sets should be divided (for each of two variables) 
  


CrossVal_Output = model.elasticNet_2dimTest(combined_Data, target_Data, ["pixel_id", "year"], ["pixel_group", "year_group"], 
                                            testGroups = [10, 6], 
                                            DataFields = ['aet__mean', 'cwd__maximum', 'cwd__mean', 'Campground',
       'FireStation_Dist', 'Airfield_Dist', 'City_Bounds', 'NPS_Bounds',
       'Lightning', 'Elev', 'Housing_Density'], outPath = 'C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/')


pixels_Years MSE Overall:  -0.0018551728711920419
pixels_Years R2 Overall:  -5.906211415448967e+33


pixels MSE Overall:  -2.8369024608413766e-05
pixels R2 Overall:  -3.822225994118679e+33


years MSE Overall:  -0.0015050008750612206
years R2 Overall:  -5.0859928587413605e+33




In [4]:
#Combined_Data with groups
CrossVal_Output[0].head()

,pixel_id,Unnamed: 0,pixel_id.1,time,aet__maximum,aet__mean,cwd__maximum,cwd__mean,Campground,FireStation_Dist,Airfield_Dist,City_Bounds,NPS_Bounds,Lightning,Elev,Housing_Density,year,pixel_group,year_group
0,135537,135537,135537,197801_198012,158.951996,61.840744,37.653000,19.579527,-1,-46,-26,-1,-1,0,91,0,1980,0,4
1,135546,135546,135546,197801_198012,133.356003,43.613838,70.831200,36.842278,-1,-30,-56,-1,-1,0,-76,0,1980,6,4
2,135569,135569,135569,197801_198012,158.940002,57.745651,43.958801,22.219458,-1,55,15,-1,-1,0,111,0,1980,0,4
3,136471,136471,136471,197801_198012,77.573128,28.880606,96.156876,51.681374,-1,-19,111,-1,-1,0,80,1,1980,4,4
4,136495,136495,136495,197801_198012,80.501877,25.742798,112.515625,53.565712,-1,-78,-60,-1,-1,0,114,0,1980,0,4


In [5]:
#target Data with groups
CrossVal_Output[1].head()

,pixel_id,value,year,pixel_group,year_group
0,135537,0,1980,0,4
1,135546,0,1980,6,4
2,135569,0,1980,0,4
3,136471,0,1980,4,4
4,136495,0,1980,0,4


In [6]:
# summary MSE and R2 for all runs, against spatially novel, temporally novel, and completely novel data
CrossVal_Output[2].head()

,Pixels_Years_MSE,Pixels_MSE,Years_MSE,Pixels_Years_R2,Pixels_R2,Years_R2
0,-0.000021,-1.221876e-07,-0.001064,-5.598074e+33,-6.017387e+33,-9.288742e+32
1,-0.002325,-1.882960e-05,-0.002196,-1.035310e+34,-2.039354e+34,-4.055591e+34
2,-0.006168,-3.031932e-05,-0.003425,-5.229212e+32,-6.696091e+33,-6.771315e+32
3,-0.000207,-3.067604e-06,-0.000351,-7.248982e+33,0.000000e+00,0.000000e+00
4,-0.000804,-1.522698e-05,-0.001033,-3.947433e+33,-6.336866e+33,-9.345976e+32


In [7]:
#List of models
CrossVal_Output[3]

[(ElasticNet(alpha=0.5, copy_X=True, fit_intercept=True, l1_ratio=0.7,
        max_iter=1000, normalize=False, positive=False, precompute=False,
        random_state=None, selection='cyclic', tol=0.0001, warm_start=False),
  -2.063837421450998e-05,
  -5.598074342713791e+33),
 (ElasticNet(alpha=0.5, copy_X=True, fit_intercept=True, l1_ratio=0.7,
        max_iter=1000, normalize=False, positive=False, precompute=False,
        random_state=None, selection='cyclic', tol=0.0001, warm_start=False),
  -0.0023249380679646325,
  -1.0353097934196502e+34),
 (ElasticNet(alpha=0.5, copy_X=True, fit_intercept=True, l1_ratio=0.7,
        max_iter=1000, normalize=False, positive=False, precompute=False,
        random_state=None, selection='cyclic', tol=0.0001, warm_start=False),
  -0.006167807380779378,
  -5.229211512509413e+32),
 (ElasticNet(alpha=0.5, copy_X=True, fit_intercept=True, l1_ratio=0.7,
        max_iter=1000, normalize=False, positive=False, precompute=False,
        random_state=None, 

In [8]:
#list of years excluded from training data for each model run
CrossVal_Output[4]

[[1986, 1988, 1991, 1997, 2004, 2012],
 [2016, 1990, 2000, 2003, 2007, 2008],
 [1984, 1994, 1998, 2010, 2011, 1981],
 [1985, 1992, 2005, 2014, 2009, 1982],
 [1995, 2001, 2002, 1980, 2013, 1983],
 [1987, 1989, 1993, 1996, 1999, 2006, 2015],
 [1986, 1988, 1991, 1997, 2004, 2012],
 [2016, 1990, 2000, 2003, 2007, 2008],
 [1984, 1994, 1998, 2010, 2011, 1981],
 [1985, 1992, 2005, 2014, 2009, 1982],
 [1995, 2001, 2002, 1980, 2013, 1983],
 [1987, 1989, 1993, 1996, 1999, 2006, 2015],
 [1986, 1988, 1991, 1997, 2004, 2012],
 [2016, 1990, 2000, 2003, 2007, 2008],
 [1984, 1994, 1998, 2010, 2011, 1981],
 [1985, 1992, 2005, 2014, 2009, 1982],
 [1995, 2001, 2002, 1980, 2013, 1983],
 [1987, 1989, 1993, 1996, 1999, 2006, 2015],
 [1986, 1988, 1991, 1997, 2004, 2012],
 [2016, 1990, 2000, 2003, 2007, 2008],
 [1984, 1994, 1998, 2010, 2011, 1981],
 [1985, 1992, 2005, 2014, 2009, 1982],
 [1995, 2001, 2002, 1980, 2013, 1983],
 [1987, 1989, 1993, 1996, 1999, 2006, 2015],
 [1986, 1988, 1991, 1997, 2004, 2012],
 

In [9]:
combined_Data.head()

,pixel_id,Unnamed: 0,pixel_id.1,time,aet__maximum,aet__mean,cwd__maximum,cwd__mean,Campground,FireStation_Dist,Airfield_Dist,City_Bounds,NPS_Bounds,Lightning,Elev,Housing_Density,year
0,135537,135537,135537,197801_198012,158.951996,61.840744,37.653000,19.579527,-1,-46,-26,-1,-1,0,91,0,1980
1,135546,135546,135546,197801_198012,133.356003,43.613838,70.831200,36.842278,-1,-30,-56,-1,-1,0,-76,0,1980
2,135569,135569,135569,197801_198012,158.940002,57.745651,43.958801,22.219458,-1,55,15,-1,-1,0,111,0,1980
3,136471,136471,136471,197801_198012,77.573128,28.880606,96.156876,51.681374,-1,-19,111,-1,-1,0,80,1,1980
4,136495,136495,136495,197801_198012,80.501877,25.742798,112.515625,53.565712,-1,-78,-60,-1,-1,0,114,0,1980


In [10]:
####################

In [9]:
q = model.elastic_YearPredictor(combined_Data, target_Data, 
                  preMasked_Data_Path = "C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Pre_Masked/",
                  outPath = "C:/Users/Python3/Documents/wildfire_FRAP_working/wildfire_FRAP/Data/Extracted_Features/Test_Preds/",  
                  year_List = [1980, 1981, 1982, 1983], 
                  DataFields = ['aet__mean', 'cwd__maximum', 'cwd__mean', 'Campground',
       'FireStation_Dist', 'Airfield_Dist', 'City_Bounds', 'NPS_Bounds',
       'Lightning', 'Elev', 'Housing_Density'],
                 mask = r"../Data/Examples/buffer/StatePoly_buf.tif")

In [10]:
q[0]

[[ElasticNet(alpha=0.5, copy_X=True, fit_intercept=True, l1_ratio=0.7,
        max_iter=1000, normalize=False, positive=False, precompute=False,
        random_state=None, selection='cyclic', tol=0.0001, warm_start=False)],
 [ElasticNet(alpha=0.5, copy_X=True, fit_intercept=True, l1_ratio=0.7,
        max_iter=1000, normalize=False, positive=False, precompute=False,
        random_state=None, selection='cyclic', tol=0.0001, warm_start=False)],
 [ElasticNet(alpha=0.5, copy_X=True, fit_intercept=True, l1_ratio=0.7,
        max_iter=1000, normalize=False, positive=False, precompute=False,
        random_state=None, selection='cyclic', tol=0.0001, warm_start=False)],
 [ElasticNet(alpha=0.5, copy_X=True, fit_intercept=True, l1_ratio=0.7,
        max_iter=1000, normalize=False, positive=False, precompute=False,
        random_state=None, selection='cyclic', tol=0.0001, warm_start=False)]]

In [11]:
q[1]

[1980, 1981, 1982, 1983]